<a href="https://colab.research.google.com/github/dannesbitt/GAIA-Agent/blob/main/GAIA_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%pip install -q langgraph langchain_openai langchain_huggingface google-cloud-speech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 5.0 MB/s eta 0:00:00


In [47]:
import os
import re
import requests
import json
from typing import TypedDict, List
from openai import OpenAI
from langgraph.graph import Graph, END
from google.colab import userdata
from urllib.parse import urlparse, parse_qs
from google.cloud import speech_v1p1beta1 as speech

# Set up OpenAI client
OPEN_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPEN_API_KEY)

# Set up Google API key and Custom Search Engine ID for web search
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')  # Custom Search Engine ID

# Define the state structure
class State(TypedDict):
    messages: List[dict]
    tool_calls: List[dict]
    final_response: str
    needs_tool_call: bool

# Define available tools, including the new "web_search" tool
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Perform a web search to find information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

def execute_tool(tool_call):
    """Executes the specified tool and returns the result."""
    if tool_call.function.name == "get_weather":
        return "It's sunny today."
    elif tool_call.function.name == "web_search":
        try:
            args = json.loads(tool_call.function.arguments)
            query = args["query"]
            api_key = GOOGLE_API_KEY
            cse_id = GOOGLE_CSE_ID
            if not api_key or not cse_id:
                return "API key or Search Engine ID not set."
            search_url = f"https://www.googleapis.com/customsearch/v1?key={api_key}&cx={cse_id}&q={query}"
            response = requests.get(search_url)
            response.raise_for_status()
            search_results = response.json()
            results = []
            for item in search_results.get('items', [])[:3]:
                title = item.get('title', 'No title')
                snippet = item.get('snippet', 'No snippet')
                results.append(f"{title}: {snippet}")
            return "\n".join(results) if results else "No results found."
        except (json.JSONDecodeError, KeyError) as e:
            return f"Error in tool call arguments: {e}"
        except Exception as e:
            return f"Error performing web search: {e}"
    return "Tool not found."

# Helper function to fetch task files (unchanged)
def fetch_task_files(task_id):
    url = f"https://agents-course-unit4-scoring.hf.space/files/{task_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        files_data = response.json()
        file_contents = {file["filename"]: file["content"] for file in files_data}
        return file_contents
    except Exception as e:
        print(f"Error fetching files for task_id {task_id}: {e}")
        return {}

# Helper function to extract YouTube video ID from a URL (unchanged)
def extract_youtube_id(url):
    parsed = urlparse(url)
    if parsed.netloc == 'www.youtube.com' and parsed.path == '/watch':
        query = parse_qs(parsed.query)
        return query.get('v', [None])[0]
    elif parsed.netloc == 'youtu.be':
        return parsed.path[1:] if parsed.path else None
    return None

# Helper function to parse SBV caption format (unchanged)
def parse_sbv(sbv_text):
    blocks = sbv_text.strip().split('\n\n')
    transcript = []
    for block in blocks:
        lines = block.split('\n')
        if len(lines) > 1:
            transcript.append(' '.join(lines[1:]))
    return ' '.join(transcript)

# Helper function to fetch YouTube transcript using YouTube Data API v3 (unchanged)
def fetch_youtube_transcript(video_id, api_key):
    if not api_key:
        print("YouTube API key not set, cannot fetch transcript.")
        return None
    try:
        list_url = f'https://www.googleapis.com/youtube/v3/captions?part=snippet&videoId={video_id}&key={api_key}'
        list_response = requests.get(list_url).json()
        if 'items' not in list_response or not list_response['items']:
            return None
        caption_id = list_response['items'][0]['id']
        download_url = f'https://www.googleapis.com/youtube/v3/captions/{caption_id}?tfmt=sbv&key={api_key}'
        sbv_text = requests.get(download_url).text
        transcript = parse_sbv(sbv_text)
        return transcript
    except Exception as e:
        print(f"Error fetching transcript for video {video_id}: {e}")
        return None

# Helper function to download audio file (unchanged)
def download_audio(url, save_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    except Exception as e:
        print(f"Error downloading audio from {url}: {e}")

# Helper function to transcribe audio using Google Cloud Speech-to-Text (unchanged)
def transcribe_audio(audio_path):
    try:
        client = speech.SpeechClient()
        with open(audio_path, 'rb') as audio_file:
            content = audio_file.read()
        audio = speech.RecognitionAudio(content=content)
        config = speech.RecognitionConfig(
            encoding=speech.RecognitionConfig.AudioEncoding.MP3,
            sample_rate_hertz=16000,  # Adjust if needed
            language_code='en-US',
        )
        response = client.recognize(config=config, audio=audio)
        transcript = ' '.join([result.alternatives[0].transcript for result in response.results])
        return transcript
    except Exception as e:
        print(f"Error transcribing audio at {audio_path}: {e}")
        return None

# Define the nodes
def input_node(state: State) -> State:
    """Fetches a question from the API, processes any additional data (files, YouTube links, audio), and constructs the initial user message."""
    if not state['messages']:
        # Add system message to inform the LLM about available tools
        state['messages'].append({
            "role": "system",
            "content": "You are an assistant that can use tools to answer questions. Available tools include 'web_search' for searching the web and 'get_weather' for weather information."
        })
        try:
            response = requests.get('https://agents-course-unit4-scoring.hf.space/random-question')
            response.raise_for_status()
            data = response.json()
#            print("Response data: ", {data})
            question = data['question']
            print("Question:", question)
            file_id = data.get('file_id', None)
            print("FileID: ", file_id)
        except Exception as e:
            print(f"Error fetching question: {e}")
            question = "What is the meaning of life?"
            file_id = None

        # Construct the user message
        user_message = f"Question: {question}\n\n"
        if file_id:
            file_contents = fetch_task_files(file_id)
            if file_contents:
                user_message += "File contents:\n"
                for filename, content in file_contents.items():
                    user_message += f"{filename}:\n{content}\n\n"

        # Check for YouTube links in the question
        youtube_urls = [url for url in re.findall(r'https?://\S+', question) if 'youtube.com' in url or 'youtu.be' in url]
        if youtube_urls:
            video_id = extract_youtube_id(youtube_urls[0])
            if video_id:
                transcript = fetch_youtube_transcript(video_id, GOOGLE_API_KEY)
                if transcript:
                    user_message += f"\n\nTranscript of the YouTube video: {transcript}"
                else:
                    user_message += "\n\nTranscript of the YouTube video: Not available"

        # Check for audio file URLs (e.g., MP3) in the question
        audio_urls = [url for url in re.findall(r'https?://\S+', question) if url.lower().endswith('.mp3')]
        if audio_urls:
            audio_url = audio_urls[0]  # Process the first audio URL only
            audio_path = '/content/audio.mp3'
            download_audio(audio_url, audio_path)
            transcript = transcribe_audio(audio_path)
            if transcript:
                user_message += f"\n\nTranscript of the audio file: {transcript}"
            else:
                user_message += "\n\nTranscript of the audio file: Not available"

        state['messages'].append({"role": "user", "content": user_message})
    state['needs_tool_call'] = False
    return state

def llm_node(state: State) -> State:
    """Calls the OpenAI LLM with the current messages and processes the response."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=state['messages'],
        tools=tools,
        tool_choice="auto",
    )
    assistant_message = response.choices[0].message
    state['messages'].append(assistant_message)

    if assistant_message.tool_calls:
        state['needs_tool_call'] = True
    else:
        state['final_response'] = assistant_message.content
        state['needs_tool_call'] = False
    return state

def tool_node(state: State) -> State:
    """Executes tool calls and appends the results to the messages."""
    assistant_message = state['messages'][-1]
    for tool_call in assistant_message.tool_calls:
        result = execute_tool(tool_call)
        state['messages'].append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id,
        })
    return state

def output_node(state: State) -> State:
    """Prints the LLM's final response."""
    print("Response:", state['final_response'])
    return state

# Create the graph
graph = Graph()

# Add nodes
graph.add_node("input", input_node)
graph.add_node("llm", llm_node)
graph.add_node("tool", tool_node)
graph.add_node("output", output_node)

# Define edges
graph.add_edge("input", "llm")
graph.add_conditional_edges(
    "llm",
    lambda state: "tool" if state['needs_tool_call'] else "output",
    {"tool": "tool", "output": "output"}
)
graph.add_edge("tool", "input")
graph.add_edge("output", END)

# Set entry point
graph.set_entry_point("input")

# Compile the graph
app = graph.compile()

# Run the workflow
initial_state = {
    "messages": [],
    "tool_calls": [],
    "final_response": "",
    "needs_tool_call": False
}
result = app.invoke(initial_state)
print("Final State:", result)

Question: The attached Excel file contains the sales of menu items for a local fast-food chain. What were the total sales that the chain made from food (not including drinks)? Express your answer in USD with two decimal places.
FileID:  None
Response: It seems like there was a mention of an Excel file, but I don't have access to any files. Could you please provide the sales data or any specific details from the Excel file? This way, I can help you calculate the total sales from food items.
Final State: {'messages': [{'role': 'system', 'content': "You are an assistant that can use tools to answer questions. Available tools include 'web_search' for searching the web and 'get_weather' for weather information."}, {'role': 'user', 'content': 'Question: The attached Excel file contains the sales of menu items for a local fast-food chain. What were the total sales that the chain made from food (not including drinks)? Express your answer in USD with two decimal places.\n\n'}, ChatCompletionMes